In [1]:
import torch
import pythreejs as pjs
import numpy as np

In [2]:
def visualize_3d(xyz, rgb=None, size=0.01, height=480, width=480):
    points_buf = pjs.BufferAttribute(array=xyz)
    geometryAttrs = {'position': points_buf}
    

    if rgb is not None:
        colors_buf = pjs.BufferAttribute(array=rgb)
        geometryAttrs['color'] = colors_buf
    
    geometry = pjs.BufferGeometry(attributes=geometryAttrs)

    material = pjs.PointsMaterial(vertexColors='VertexColors', size=size)
    pointCloud = pjs.Points(geometry=geometry, material=material)

    pythreejs_camera = pjs.PerspectiveCamera(    
        up=[1, 0, 1],
        children=[pjs.DirectionalLight(color='blue', intensity=0.5)])

    pythreejs_camera.rotateX(np.pi/4)
    pythreejs_camera.position = (200., 1000., 100.)

    scene = pjs.Scene(children=[
                    pointCloud,
                    pythreejs_camera,
                    pjs.AmbientLight(color='#777777')])
    
    axes = pjs.AxesHelper(size=3)
    scene.add(axes)
        
    control = pjs.OrbitControls(controlling=pythreejs_camera)
    renderer = pjs.Renderer(camera=pythreejs_camera, 
                        scene=scene, 
                        width=width,
                        height=height,
                        preserveDrawingBuffer=True,
                        controls=[control])
    
    return renderer

In [3]:
pred = np.load('/home/lcw/workspace/0000000000_depth_0_pred.npz', allow_pickle=True)
pred = pred['velodyne_depth']
gt = np.load('/home/lcw/workspace/0000000000.npz')
gt = gt['velodyne_depth']

i,j = np.indices((pred.shape))

points = np.vstack((i.flatten(),j.flatten(),pred.flatten())).T
gt_points = np.vstack((i.flatten(),j.flatten(),gt.flatten())).T

In [4]:
################################################################################################
################----------visualize_predicted_depth__&&__ground_truth----------#################
################################################################################################

In [5]:
visualize_3d(points)
visualize_3d(gt_points)

/home/lcw/.local/lib/python3.8/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='blue', intensity=0.5, position=(0.0, 0.0, …

In [6]:
def compute_depth_errors(gt, pred):
    valid_mask = (gt > 0) & (pred > 0)
    
    # 마스크를 적용하여 0이 아닌 요소만을 선택
    gt_valid = gt[valid_mask]
    pred_valid = pred[valid_mask]

    abs_rel = torch.mean(torch.abs(gt_valid - pred_valid) / gt_valid)

    sq_rel = torch.mean((gt_valid - pred_valid) ** 2 / gt_valid)
    
    rmse = (gt_valid - pred_valid) ** 2
    rmse = torch.sqrt(rmse.mean())

    rmse_log = (torch.log(gt) - torch.log(pred)) ** 2
    rmse_log = torch.sqrt(rmse_log.mean())


    delta = torch.max((gt/ pred), (pred / gt))
    a1 = (delta < 1.25     ).float().mean()
    a2 = (delta < 1.25 ** 2).float().mean()
    a3 = (delta < 1.25 ** 3).float().mean()

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3


In [7]:
gt = torch.from_numpy(gt)
pred = torch.from_numpy(pred)
print(compute_depth_errors(gt,pred))

(tensor(0.), tensor(0.), tensor(0.), tensor(nan), tensor(0.0413), tensor(0.0413), tensor(0.0413))
